We will train on the first 50% of article ids (as determined by the get_id_dict function).

In [1]:
import os, sys
sys.path.append("../utils")
from constants import *
from helper import get_id_dict

import numpy as np
import pandas as pd
import torch as t
from sklearn.linear_model import LogisticRegression

import pickle
from typing import List
from tqdm import tqdm, trange


n_subset_dict = {
    "newsroom": 60,
    "summeval": 100
}
n_example_dict = {
    "newsroom": 7,
    "summeval": 16
}

def get_label(scores: List[int]) -> int:
    s1, s2 = scores
    if s1 > s2: return 1
    else: return 0

In [2]:
dataset = "newsroom"
id_dict = get_id_dict(dataset)

article_ids = [k for k in id_dict.keys()]
train_ids = article_ids[:len(article_ids)//2]
test_ids = article_ids[len(article_ids)//2:]
prompts = pd.read_json(f"{gdrive_path}/model_harvesting/prompts_short/{dataset}-mine-compare-1.jsonl", orient="records", lines=True)
train_ixs = prompts.loc[prompts["article_id"].isin(train_ids)].index
test_ixs = prompts.loc[prompts["article_id"].isin(test_ids)].index

for model_name in ["llama2", "llama3", "mistral"]:
    for aspect in tqdm(["consistency", "coherence", "fluency", "relevance"], desc=model_name):
        prompts["label"] = prompts[aspect].apply(get_label)
        c1 = t.load(f"{gdrive_path}/model_harvesting/{model_name}/activations_short/{dataset}_{aspect}_1.pt")
        c2 = t.load(f"{gdrive_path}/model_harvesting/{model_name}/activations_short/{dataset}_{aspect}_2.pt")
        data = c1 - c2
        y_train, y_test = prompts.loc[train_ixs, "label"].values, prompts.loc[test_ixs, "label"].values
        X_train, X_test = data[train_ixs], data[test_ixs]
        perm = t.randperm(len(X_train))
        X_train, y_train = X_train[perm], y_train[perm]
        lr = LogisticRegression(max_iter=1000)
        lr.fit(X_train, y_train)

        comparisons = np.zeros((n_subset_dict[dataset]//2, n_example_dict[dataset], n_example_dict[dataset])) + 0.5
        for subset_ix, subset_id in enumerate(test_ids):
            _condition = prompts["article_id"] == subset_id
            example_ids = id_dict[subset_id]
            for example1 in example_ids:
                example1_ix = example_ids.index(example1)
                for example2 in example_ids:
                    example2_ix = example_ids.index(example2)
                    if example1 == example2: continue
                    condition = _condition & (prompts["system_id"].apply(lambda x: x==[example1, example2]))
                    ix = prompts.loc[condition].index
                    if len(ix) == 0:
                        condition = _condition & (prompts["system_id"].apply(lambda x: x==[example2, example1]))
                        ix = prompts.loc[condition].index
                    p_s1, p_s2 = lr.predict_proba(data[[ix[0]], :]).flatten()
                    comparisons[subset_ix, example1_ix, example2_ix] = p_s1
                    comparisons[subset_ix, example2_ix, example1_ix] = p_s2
        outfile = open(f"../comparison_matrices/{model_name}/activations-short-{dataset}-{aspect}.pickle", "wb")
        pickle.dump(comparisons, outfile); outfile.close()

mistral: 100%|██████████| 4/4 [00:27<00:00,  6.93s/it]
